In [4]:
!!pip install -U pymoo

['Requirement already satisfied: pymoo in c:\\users\\user\\appdata\\local\\programs\\python\\python312\\lib\\site-packages (0.6.1.5)',
 'Requirement already satisfied: numpy>=1.19.3 in c:\\tools\\manim\\lib\\site-packages (from pymoo) (2.2.3)',
 'Requirement already satisfied: scipy>=1.1 in c:\\tools\\manim\\lib\\site-packages (from pymoo) (1.15.2)',
 'Requirement already satisfied: matplotlib>=3 in c:\\users\\user\\appdata\\local\\programs\\python\\python312\\lib\\site-packages (from pymoo) (3.10.0)',
 'Requirement already satisfied: autograd>=1.4 in c:\\users\\user\\appdata\\local\\programs\\python\\python312\\lib\\site-packages (from pymoo) (1.8.0)',
 'Requirement already satisfied: cma>=3.2.2 in c:\\users\\user\\appdata\\local\\programs\\python\\python312\\lib\\site-packages (from pymoo) (4.2.0)',
 'Requirement already satisfied: alive-progress in c:\\users\\user\\appdata\\local\\programs\\python\\python312\\lib\\site-packages (from pymoo) (3.2.0)',
 'Requirement already satisfied:

In [1]:
import pandas as pd
import os
import joblib
import numpy as np
from pymoo.core.problem import Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.optimize import minimize
from pymoo.termination import get_termination
#from pymoo.samplers import Sampling
#from pymoo.factory import Sampling
from pymoo.operators.sampling.lhs import LHS
from sklearn.preprocessing import MinMaxScaler

In [6]:
# Cargar datos
# Directorio y carga
directorio_actual = os.getcwd()
archivo_csv = os.path.join(directorio_actual, "DPta_CarEngMxI&Q_6.0md1.csv")
data= pd.read_csv(archivo_csv, encoding='latin1', sep=';')


In [7]:
# Variables independientes (de diseño)
variables = ['F_Alim', 'T_Alim', 'T_Vap', 'P_Vap', 'T_Acond', 't_ret',
             'H_Pelt', 'T_Pelt', 'M_Vap', 'H_Add']

In [8]:
# Variables dependientes (targets)
objetivos = ['Q_Vap', 'Fn', 'Prodt', 'PDI', 'DGM']


In [9]:
# Escalamiento
scaler = MinMaxScaler()
X = scaler.fit_transform(data[variables])
Y = data[objetivos].values

In [10]:
# Dataset combinado
dataset = pd.DataFrame(X, columns=variables)
dataset[['Q_Vap', 'Fn', 'Prodt', 'PDI', 'DGM']] = Y

In [11]:
# -------------------------
# DEFINICIÓN DEL PROBLEMA
# -------------------------

class OptimizarProceso(Problem):
    def __init__(self):
        super().__init__(n_var=len(variables),
                         n_obj=3,  # Minimizar Q_Vap, Minimizar Fn, Maximizar Prodt
                         n_constr=3,  # Restricciones: Fn<5, PDI>91, 3900<=DGM<=4100
                         xl=0.0, xu=1.0)  # Todos los valores están escalados

    def _evaluate(self, X, out, *args, **kwargs):
        f1, f2, f3 = [], [], []
        g1, g2, g3 = [], [], []

        for x in X:
            # Buscar punto más cercano del dataset
            dists = np.linalg.norm(dataset[variables].values - x, axis=1)
            idx = np.argmin(dists)
            row = dataset.iloc[idx]

            # Objetivos
            f1.append(row['Q_Vap'])      # minimizar
            f2.append(row['Fn'])         # minimizar
            f3.append(-row['Prodt'])     # maximizar (=> minimizar negativo)

            # Restricciones (se deben satisfacer: g ≤ 0)
            g1.append(row['Fn'] - 5)
            g2.append(91 - row['PDI'])
            g3.append(max(0, 3900 - row['DGM']) + max(0, row['DGM'] - 4100))

        out["F"] = np.column_stack([f1, f2, f3])
        out["G"] = np.column_stack([g1, g2, g3])


In [12]:
# -------------------------
# EJECUTAR OPTIMIZACIÓN
# -------------------------
problem = OptimizarProceso()

algorithm = NSGA2(pop_size=100)

termination = get_termination("n_gen", 50)  # 50 generaciones

res = minimize(problem,
               algorithm,
               termination,
               seed=42,
               save_history=True,
               verbose=True)


n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |      100 |      8 |  0.000000E+00 |  1.751479E+02 |             - |             -
     2 |      200 |     23 |  0.000000E+00 |  0.000000E+00 |  0.1935483871 |         ideal
     3 |      300 |     48 |  0.000000E+00 |  0.000000E+00 |  0.2911459437 |         ideal
     4 |      400 |     58 |  0.000000E+00 |  0.000000E+00 |  0.1153846154 |         nadir
     5 |      500 |     98 |  0.000000E+00 |  0.000000E+00 |  0.0795454545 |         ideal
     6 |      600 |    100 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     7 |      700 |    100 |  0.000000E+00 |  0.000000E+00 |  0.0714285714 |         ideal
     8 |      800 |    100 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
     9 |      900 |    100 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f
    10 |     1000 |    100 |  0.000000E+00 |  0.000000E+00 |  0.000000E+00 |             f

In [ ]:
# -------------------------
# RESULTADOS
# -------------------------

# Desescalar los valores para entender resultados reales
resultados = []
for x in res.X:
    dists = np.linalg.norm(dataset[variables].values - x, axis=1)
    idx = np.argmin(dists)
    row = dataset.iloc[idx]
    resultados.append(row)

df_resultados = pd.DataFrame(resultados)
df_resultados['Q_Vap'] = df_resultados['Q_Vap'].round(2)
df_resultados['Fn'] = df_resultados['Fn'].round(2)
df_resultados['Prodt'] = df_resultados['Prodt'].round(2)
df_resultados['PDI'] = df_resultados['PDI'].round(2)
df_resultados['DGM'] = df_resultados['DGM'].round(2)

# Mostrar mejores soluciones
#print("\n🏆 Mejores soluciones óptimas no dominadas (cumpliendo restricciones):")
#print(df_resultados.head(10))


🏆 Mejores soluciones óptimas no dominadas (cumpliendo restricciones):
        F_Alim    T_Alim     T_Vap     P_Vap   T_Acond     t_ret    H_Pelt  \
502   0.412480  0.467780  0.851562  0.651254  0.721831  0.539799  0.569697   
2490  0.339141  0.295943  0.125000  0.550006  0.000000  0.539799  0.454545   
3163  0.666126  0.532220  0.812500  0.662504  0.686620  0.931822  0.212121   
2490  0.339141  0.295943  0.125000  0.550006  0.000000  0.539799  0.454545   
122   0.294571  0.427208  0.867188  0.656879  0.704225  0.488665  0.575758   
3176  0.768639  0.312649  0.750000  0.662504  0.633803  0.931822  0.121212   
154   0.107374  0.414797  0.750000  0.550006  0.669014  0.539799  0.400000   
168   0.037682  0.496420  0.750000  0.662504  0.607394  0.539799  0.393939   
105   0.313614  0.501193  0.750000  0.648442  0.708627  0.539799  0.493939   
193   0.764587  0.431981  0.750000  0.640004  0.535211  0.700017  0.327273   

        T_Pelt     M_Vap     H_Add    Q_Vap    Fn  Prodt    PDI     DG

In [14]:
# Desescalar variables de entrada (F_Alim, T_Alim, ..., H_Add)
variables_escaladas = df_resultados[variables].values
variables_dese = scaler.inverse_transform(variables_escaladas)

# Reemplazar en el DataFrame
df_resultados[variables] = variables_dese

# Redondear todos los valores relevantes
for col in df_resultados.columns:
    df_resultados[col] = df_resultados[col].round(2)

# Mostrar y guardar los resultados corregidos
print("\n🏆 Mejores soluciones óptimas no dominadas (desescaladas y válidas):")
print(df_resultados.head(10))





🏆 Mejores soluciones óptimas no dominadas (desescaladas y válidas):
      F_Alim  T_Alim   T_Vap   P_Vap  T_Acond  t_ret  H_Pelt  T_Pelt  M_Vap  \
502    25.25   25.75  173.25  109.00    80.00   95.0   12.28   28.05   2.68   
2490   23.44   22.15  150.00  100.00    39.00   95.0   11.90   25.30   0.75   
3163   31.51   27.10  172.00  110.00    78.00  118.0   11.10   30.10   3.14   
2490   23.44   22.15  150.00  100.00    39.00   95.0   11.90   25.30   0.75   
122    22.34   24.90  173.75  109.50    79.00   92.0   12.30   30.43   2.37   
3176   34.04   22.50  170.00  110.00    75.00  118.0   10.80   25.60   3.50   
154    17.72   24.64  170.00  100.00    77.00   95.0   11.72   30.04   1.77   
168    16.00   26.35  170.00  110.00    73.50   95.0   11.70   32.40   1.48   
105    22.81   26.45  170.00  108.75    79.25   95.0   12.03   29.85   2.35   
193    33.94   25.00  170.00  108.00    69.40  104.4   11.48   30.66   2.94   

      H_Add    Q_Vap    Fn  Prodt    PDI     DGM  
502    9.5

In [15]:
# Guardar resultados
df_resultados.to_csv("optimizacion_nsga2_resultados.csv", index=False)

#### Análisis Comparativo de Resultados entre Algoritmo Basado en Aprendizaje Profundo (DCQL) y Optimización Evolutiva (NSGA-II) utilizando una Función de Recompensa Unificada

In [17]:
import pandas as pd

In [18]:
# Paso 1: Leer los resultados generados por NSGA-II (ya deben estar desescalados)
df_nsga = pd.read_csv("optimizacion_nsga2_resultados.csv")

In [19]:
# Paso 2: Definir la misma función de recompensa que usaste en DCQL
def calcular_recompensa(row):
    prodt = row['Prodt']
    q_vap = row['Q_Vap']
    fn = row['Fn']
    pdi = row['PDI']
    dgm = row['DGM']

    penalizacion_fn = max(0, (fn - 5) * 20)
    penalizacion_qvap = q_vap * 0.01
    penalizacion_pdi = 0 if pdi > 91 else 1000
    penalizacion_dgm = 0 if 3900 <= dgm <= 4100 else 1000

    recompensa = prodt * 10 - penalizacion_fn - penalizacion_qvap - penalizacion_pdi - penalizacion_dgm
    return round(recompensa, 4)

In [25]:
# Aplicar la recompensa a cada fila
df_resultados['score'] = df_resultados.apply(calcular_recompensa, axis=1)

In [26]:
# Obtener la mejor fila según el mayor score
mejor_fila = df_resultados.loc[df_resultados['score'].idxmax()]

In [28]:
# Verifica si mejor_fila es una Serie (una sola fila seleccionada)
if isinstance(mejor_fila, pd.Series):
    # 📌 Mostrar solo los indicadores clave en horizontal
    print("🔹 Mejores indicadores clave:")
    print(mejor_fila[['Prodt', 'Q_Vap', 'Fn', 'PDI', 'DGM']].to_frame().T)

    # 📌 Mostrar toda la fila completa en horizontal
    print("\n🔹 Toda la fila completa:")
    print(mejor_fila.to_frame().T)

else:
    # Si es un DataFrame con una sola fila
    print("🔹 Mejores indicadores clave:")
    print(mejor_fila[['Prodt', 'Q_Vap', 'Fn', 'PDI', 'DGM']])

    print("\n🔹 Toda la fila completa:")
    print(mejor_fila)


🔹 Mejores indicadores clave:
     Prodt    Q_Vap    Fn   PDI     DGM
502  96.61  1114.88  2.25  92.4  4028.0
502  96.61  1114.88  2.25  92.4  4028.0

🔹 Toda la fila completa:
     F_Alim  T_Alim   T_Vap  P_Vap  T_Acond  t_ret  H_Pelt  T_Pelt  M_Vap  \
502   25.25   25.75  173.25  109.0     80.0   95.0   12.28   28.05   2.68   
502   25.25   25.75  173.25  109.0     80.0   95.0   12.28   28.05   2.68   

     H_Add    Q_Vap    Fn  Prodt   PDI     DGM     score  
502   9.59  1114.88  2.25  96.61  92.4  4028.0  954.9512  
502   9.59  1114.88  2.25  96.61  92.4  4028.0  954.9512  
